# Workshop 3 - Quality control for sequencing data

Suppose you had a biological question of interest that could be solved by sequencing. You've received some funding to perform a sequencing run (lucky you!). The sample is run in a sequencing machine (you know a few kinds of sequencers from the sequencing technologies lecture) and you are delivered a file containing all of your reads. It's possible that there were problems, either with the sequencing reaction or with the starting materials. Before you perform your analysis and report your findings, you want to be sure that the data you have is of good quality. This is where quality control (QC) comes in.

This week you will write your own code to perform QC on some real sequencing data.

<br>

## Task 0 - Getting set up for plotting in python

There are three python libraries that we will be using to produce figures -  `numpy`, `pandas` and `matplotlib`.  

All three are already installed on SWAN. `numpy` and `pandas` are used for data manipulation and processing, while `matplotlib` is used to plot and visualise your data.

Check that they are installed by importing them.


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# You can install them on your personal machine with the following conda or pip commands.

# conda install numpy pandas
# pip install numpy pandas
# pip install matplotlib
# conda install -c conda-forge matplotlib

<br>

If you are new to `pandas`, this tutorial below will help. It is also available under the additional resources module on the LMS.
> http://pandas.pydata.org/pandas-docs/stable/getting_started/10min.html

We use the `matplotlib` library to create simple plots. You can learn more about `matplotlib` using the link below. The link is also available on the additional resources module on the LMS.
>https://matplotlib.org/tutorials/introductory/usage.html#sphx-glr-tutorials-introductory-usage-py

The following tutorial shows how `pandas`, `numpy` and `matplotlib` can be used together.
> https://www.kaggle.com/chats351/introduction-to-numpy-pandas-and-matplotlib

<br>

## Task 1 - Reading in the read set

First, we read in some sample reads. The reads in `ERR024571_selected.fastq` are a selection of 800 reads from the ERR024571 read set.

Note that converting a readset into a list of `skbio` objects makes it easier to handle.

The data originates from a *Yersinia enterocolitica* sample, a bacterial species that causes yersiniosis in humans.

<br>


In [ ]:
import skbio

In [ ]:
fname = 'ERR024571_selected.fastq' 
registry = skbio.io.read(fname, format = 'fastq', phred_offset = 33)

readset = list(registry)

<br>
We can use the len function to find the number of reads in the readset.

In [ ]:
len(readset)

And to find the length of the first read in the readset.

In [ ]:
len(readset[0])


More can be read about these skbio objects here:
> http://scikit-bio.org/docs/0.5.0/generated/skbio.sequence.DNA.html#skbio.sequence.DNA

<br>

## Task 2 - Compute per-sequence quality scores

The per-sequence quality is simply the average of the base quality scores in a sequence.

In [ ]:
# Display the quality scores for the first 10 bases of the first read.
r1_meta = readset[0].positional_metadata
r1_meta.head(10)

### Crash course on `pandas` and `numpy`

The output of the code above is a data frame object. Most data can be stored in data frames and there are many functions to process them available in the `pandas` library. 

The `numpy` library stores data in `numpy` arrays, which can range from being one-dimensional vectors or multi-dimensional matrices. A `pandas` data frame is equivalent to a 2-dimensional `numpy` array with named rows and columns. While `numpy` arrays are more memory efficient, `pandas` dataframes have other advantages such as named rows and columns.

We will first try out a few indexing operations.


In [ ]:
import pandas as pd
import numpy as np

# All of these indexes give the same results. Think of the data frame as a 2-D matrix.
# Here we retrieve data using .iloc

print(r1_meta.iloc[:3,])
print(r1_meta.iloc[0:3, 0:1])
print(r1_meta.iloc[0:3, :])
print(r1_meta.iloc[0:3,])

Above we use positional information along the data frame (i.e. rows 0:3 and column 0:1). 

Instead, we can index rows and columns using *row names* and *column names* with `pandas`. 
In the example below, 0:2 refers to the rows with names '0', '1', and '2' and the column with the name 'quality'. 



In [ ]:
# Here we retrieve data using names with loc. 0, 1, and 2 are row names rather than indexes.
print(r1_meta.loc[0:2, 'quality'])

Because we selected just one column a one-dimensional vector was returned. 

When using `pandas` this object is called a `pandas` series, which is equivalent to a one-dimensional `numpy` array.

In [ ]:
print(type(r1_meta.loc[0:2, 'quality']))
print(type(np.asarray(r1_meta.loc[0:2, 'quality'])))

<br>

If we retrieve the entire `quality` column, we can subset it obtain the base quality at any position.

In [ ]:
# Retrieve the base quality at position 3. Remember that Python is zero indexed!
r1_qualities = r1_meta.loc[:, 'quality']
r1_qualities[2]

<br>

### Now to compute the quality of a sequence

We can now write a function to compute the sequence quality of a given sequence using a for loop and the `.positional_metadata` method.

In [ ]:
def read_quality_with_loop(read):
    """
    Compute the quality of a given read (DNA sequence).
    This is the average of base qualities.
    Return a single floating point number.
    """



In [ ]:
print(read_quality_with_loop(readset[0])) # should give 27.36
print(read_quality_with_loop(readset[2])) # should give 26.35

<br>

Using functions from within the `numpy` library can be simpler to write and run considerably faster. 

Try computing the sequence quality of a given sequence **without** a for loop, using the `np.mean()` function.

In [ ]:
def read_quality(read):
    """
    Compute the quality of a given read (DNA sequence).
    This is the average of base qualities.
    Return a single number representing the quality
    """
    

In [ ]:
print(read_quality(readset[0])) # should also give 27.36

<br>

## Task 3 - Compute and visualise the per-sequence quality distribution

We will compute per-sequence qualities for all reads and compute a sequence quality histogram. A histogram visualises the distribution of data by binning the data values into intervals of equal widths and visualising the frequency of values in each bin. 

First, you should divide the range of values into equal intervals. These intervals should be non-overlapping integers. Here we would like bins with a width of 1.  
i.e. For a given integer \$i\$, the interval (bin) should be \$[i, i + 1)\$. This means including \$i\$ but excluding \$i + 1\$.


Second, you need to count up the frequency of values that fall into each bin. We can represent this histogram data as a dictionary. Keys in the dictionary will be \$i\$ and the values will be the total number of quality scores that fall in the interval \$[i, i + 1)\$.


When defining your bins, the `np.floor()` function can be be used to round down quality scores to the nearest integer.

In [ ]:
print(np.floor(30.01))
print(np.floor(30.99))

In [ ]:
def read_histogram_dict(reads):
    """
    Compute a dictionary representing the histogram
    Keys are the starting point of the interval
    Values are the number of items in the interval
    """


In [ ]:
read_histogram_dict(readset[0:4]) # should give {27.0: 2, 28.0: 1, 26.0: 1}

<br>

Now that we have the histogram data as a dictionary, let's get to plotting! 


In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

hist_data = read_histogram_dict(readset)
plt.scatter(hist_data.keys(), hist_data.values())
plt.xlabel('Phred quality score')
plt.ylabel('Frequency')
plt.show()

<br>

Alternatively, you could directly use the `plt.hist()` function in `matplotlib`.

This function takes the data values you want to bin, and the intervals you want to put them in. But you should provide the bins in order with `sorted()`.


In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

# Get the read qualities
read_qualities = [read_quality(r) for r in readset]

# Define the intervals [i,i+1)
distribution_keys = [np.floor(r) for r in read_qualities]

# Include the upper bound of the last interval
# distribution_keys.append(max(sorted(distribution_keys))+1)

# Plot
plt.hist(read_qualities, sorted(distribution_keys))
plt.xlabel('Phred quality score')
plt.ylabel('Frequency')
plt.show()

<br>

We can also change the colour, overlay lines, and edit the appearance of the plot. 

Change some of the arguments (`fontsize`, `labelpad` or `color`) below to see what effect they have.

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

# Plot
plt.hist(read_qualities, sorted(distribution_keys), color="skyblue")
plt.xlabel('Phred quality score', fontsize=14)
plt.ylabel('Frequency', fontsize=14, labelpad=12)
plt.axvline(x=20,linewidth=1, color='r')
plt.show()

<br>

You can also make other plot types, like boxplots.

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

# Plot
plt.boxplot(read_qualities, vert=False)
plt.xlabel('Phred quality score', fontsize=14, labelpad=12)
plt.yticks([])
plt.show()

<br>

The documentation for these functions can be found here:
> https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.scatter.html

> https://matplotlib.org/stable/api/_as_gen/matplotlib.pyplot.hist.html

> https://matplotlib.org/3.1.1/api/_as_gen/matplotlib.pyplot.boxplot.html

A list of colour names can be found here:
> https://matplotlib.org/stable/gallery/color/named_colors.html

<br>

## Task 4 - Using FastQC for quality control

The above analysis and more can be performed using the FastQC tool. 

FastQC is installed on SWAN.
If you wish to install FastQC on a (Mac or Linux) personal device, use the conda command `conda install -c bioconda fastqc`.


The documentation for FastQC can be found here:
> https://www.bioinformatics.babraham.ac.uk/projects/fastqc/

**Please execute the command below to run FastQC.**

In [ ]:
# Run FastQC on the ERR024571_1 dataset.
!fastqc -o ./ ERR024571_1.fastq.gz

The FastQC report can be viewed by right clicking the file ERR024571_1_fastqc.html and selecting *Open in New Browser Tab*


###  What does this all mean?

The report provides information about several QC metrics.

You should refer to this documentation to help you figure out what some of the metrics mean: 
>https://dnacore.missouri.edu/PDF/FastQC_Manual.pdf


There is also a FastQC report explanation video on the LMS in the week 3 module.

<br>


### Try answering some of these questions:

<br>

1. How many reads are in the read set?

2. How long are the reads?

3. Are all the reads the same length?

4. How are the quality scores distributed across the length of the reads?

5. Are there any overrepresented sequences?

6. What is the likely source of the overrepresented sequences?

7. Overall, is this read set of acceptable quality?

<br>

### Some comments about quality control

<br>

What we have covered in this workshop is just one layer of quality control. 

Further QC would need to be done, depending on the type of analysis. QC should form an important component of all analyses. Ultimately, it can save a lot of time and effort!



FastQC outputs a report for an individual FASTQ file. If you are dealing with multiple read sets in your own projects, MultiQC provides a summary of all FastQC reports. See https://multiqc.info/ and have a look at the example reports.

<br>

`Workshop developed by Steven Morgan, Dr Dieter Bulach, Dharmesh Bhuva and Holly Whitfield.`